# GEMdataset

> This package will hold the GxExM dataset.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

In [ ]:
#| export
def collect_snps(method):
    """
    Input
        method: a Path(PosixPath) to directory containing npy arrays for each chr
    Output
        tuple(hybrid ids, snp matrix)
    """
    for c,chr in enumerate(method.iterdir()):
        if c == 0:
            strains,snp_data = np.load(chr,allow_pickle=True)
        else:
            strains, snps = np.load(chr, allow_pickle=True)
            snp_data = np.vstack((snp_data,snps))

            return strains,snp_data

In [ ]:
#| export
class WT():
    """
    A class which will hold the weather data for the entire dataset for training purposes
    
    init
        weather_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, weather_data, testYear):
        
        self.Te = weather_data.iloc[([str(testYear) in x for x in weather_data['Year']]),:].reset_index()
        self.Tr = weather_data.iloc[([str(testYear) not in x for x in weather_data['Year']]),:].reset_index()
            
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)
            
    def setup_scaler(self):
        ss = StandardScaler()
        ss.fit(self.Tr.select_dtypes('float'))
        self.scaler = ss
            
    def scale_data(self, df):
        fd = df.select_dtypes('float')
        fs = self.scaler.transform(fd)
        df[fd.columns] = fs

In [ ]:
#| export
class YT():
    """
    A class which will hold the yield data for the entire dataset for training purposes
    
    init
        yield_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, yield_data, testYear):

        self.Te = yield_data.iloc[([str(testYear) in x for x in yield_data['Env']]),:].reset_index()
        self.Tr = yield_data.iloc[([str(testYear) not in x for x in yield_data['Env']]),:].reset_index()

        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

    def setup_scaler(self):
        ss = StandardScaler()
        ss.fit(np.array(self.Tr['Twt_kg_m3']).reshape(-1,1))
        self.scaler = ss

    def scale_data(self,df):
        ya = np.array(df['Twt_kg_m3']).reshape(-1,1)
        ys = self.scaler.transform(ya)
        df['scaled_yield'] = ys

    def plot_yields(self):

        plt.hist(self.Tr['scaled_yield'],density=True, label='Train',alpha=.5,bins=50)
        plt.hist(self.Te['scaled_yield'],density=True, label='Test',alpha=.5,bins=50)
        plt.legend()
        plt.show()

In [ ]:
#| export
class GEM():
    """
    init
        split -> the year that will be designated as the test split
    """
    def __init__(self, split):
        self.split = str(split)
        self.Y = None
        self.W = None
        self.SNP = None

In [ ]:
#| export
class GemDataset():
    """
    Pytorch Dataset which can be used with dataloaders for simple batching during training loops
    """
    def __init__(self,W,Y,G):
        self.W = W
        self.SNP = G
        self.Y = Y
        
    def __len__(self): return self.Y.shape[0]

    def __getitem__(self,idx):
        #keys to access SNPs and Weather
        hybrid = self.Y.iloc[idx,:]['Hybrid']
        env = self.Y.iloc[idx,:]['Env']

        #values for the model training
        target = self.Y.iloc[idx,:]['scaled_yield']
        genotype = self.SNP[1][:, np.where(self.SNP[0]==hybrid)[0][0]]
        weather = np.array(self.W.loc[self.W['Env'] == env].select_dtypes('float'))

        return target, genotype, weather        

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()